In [ ]:
import os
import json
import xmltodict
from tqdm import tqdm
import shutil
import numpy as np
import pandas as pd
import openslide
import tifffile
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

import PIL.Image
PIL.Image.MAX_IMAGE_PIXELS = None

In [ ]:
dataDir = '/path/to/whole-slide-images/'
saveDir = '/workdir/thumbs/'
ext = '.svs'

if not os.path.exists(saveDir):
    os.makedirs(saveDir)

names = [name for name in os.listdir(dataDir) if name[-len(ext):]==ext]
print(len(names))

# Get thumbnails

In [ ]:
if True:
    for ifname, fname in enumerate(names):
        if ext in fname:
            sname = saveDir + fname[:-len(ext)] + '.tiff'
            if not os.path.isfile(sname):
                print(f'{ifname} of {len(names)}', fname, end='\t')
                try:
                    slide = openslide.open_slide(dataDir + fname)
                    x, y = slide.dimensions
                    print((x, y), end='\t')
    
                    print('Generating metadata')
                    d = dict(slide.properties)
                    slide.close()
                        
                    d.update({'x': x, 'y': y})
                    
                    with open(saveDir + 'magnification_dimensions_%s.json' % fname[:-len(ext)], 'w') as outfile:
                        outfile.write(json.dumps(d))

                    if not os.path.isfile(sname):
                        print('Generating thumbnail')
                        tifffile.imwrite(sname, tifffile.imread(dataDir + fname)[::50, ::50, :])
                    
                    print()
                except Exception as exception:
                    print('ERROR:', exception)

# Generate ROIs

In [ ]:
def prepInfoJSON(tdatapath, model, fname, eps=50, min_samples=500, selobj=None, f=1.5, efx=0.35, efy=0.35, manshift=[0,0,0,0],
                 thumb_factor=100, further_thumb_downsample_facator=3, pen_marked=0, coords=None, thumbExt='tiff', a=80, b=225):

    id = fname[:-len(f'.{thumbExt}')]
    
    img_RGB_high_res = plt.imread(tdatapath + fname)
    
    thumb_factor *= further_thumb_downsample_facator
    img_RGB_high_res  = img_RGB_high_res[::further_thumb_downsample_facator, ::further_thumb_downsample_facator, :]
    
    print(img_RGB_high_res.shape, end='\t')

    v = img_RGB_high_res[:, :, :3].copy()
    v = v.mean(axis=2)
    
    vc = v.copy()
    v[vc<a] = 255/255
    v[vc>b] = 255/255
    v[(vc>=a/255) & (vc<=b/255)] = 0
    v = pd.DataFrame(v.T).replace({255/255: np.nan}).stack().dropna().index.to_frame().values
    
    fx = img_RGB_high_res.shape[1]/thumb_factor
    fy = img_RGB_high_res.shape[0]/thumb_factor
    fig, axs = plt.subplots(1, 2, figsize=(fx*15, fy*4))

    axs[0].imshow(img_RGB_high_res[:, :, :3], origin='lower')
    axs[0].set_xticks([])
    axs[0].set_xticklabels([])
    axs[0].set_yticks([])
    axs[0].set_yticklabels([])
    
    np.random.seed(0)
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(v)
    labels = db.labels_
    vco = pd.Series(labels).value_counts()
    
    axs[1].scatter(v.T[0], v.T[1], c=labels, s=1)
    for l in vco.index:
        if l!=-1:
            axs[1].text(np.median(v.T[0][np.where(labels==l)[0]]),
                     np.median(v.T[1][np.where(labels==l)[0]]),
                     l, va='center', ha='center', fontsize=20)

    existing = [f for f in os.listdir(tdatapath) if ('.oid' in f) and (id in f)]
    for f in existing:
        os.remove(tdatapath + f)
    
    objects = sorted(list(set(vco.to_dict().keys()).difference([-1])))
    res = dict()
    for l in objects:
        res[l] = dict()
        res[l]['model'] = model
        res[l]['id'] = id
        for i in range(2):
            res[l][i] = dict()
            se_temp = pd.Series(v.T[i][np.where(labels==l)[0]])
            a, b = se_temp.quantile(0.05), se_temp.quantile(0.95)
            sp = b - a

            ef = efx if i==0 else efy
            sa, sb = (manshift[0], manshift[1]) if i==0 else (manshift[2], manshift[3])
            a -= (ef+sa)*sp
            b += (ef+sb)*sp

            d = img_RGB_high_res.shape[1 if i==0 else 0]
            a, b = a/d, b/d - a/d
            a = max(a, 0)
            if a + b > 1:
                b = 1 - a
            res[l][i]['location'] = a
            res[l][i]['size'] = b
            
        x1 =  res[l][0]['location']*img_RGB_high_res.shape[1]
        x2 =  x1 + res[l][0]['size']*img_RGB_high_res.shape[1]
        y1 =  res[l][1]['location']*img_RGB_high_res.shape[0]
        y2 =  y1 + res[l][1]['size']*img_RGB_high_res.shape[0]
        axs[1].plot([x1, x1, x2, x2, x1], [y1, y2, y2, y1, y1], linewidth=0.5, c='crimson')
        
        with open(tdatapath + id + f'.oid{l}.json', 'w') as outfile:
            outfile.write(json.dumps(res[l]))
    
    axs[1].set_xticks([])
    axs[1].set_xticklabels([])
    axs[1].set_yticks([])
    axs[1].set_yticklabels([])

    axs[0].set_aspect('equal')
    axs[1].set_aspect('equal')

    axs[0].set_ylim(axs[0].get_ylim()[::-1])
    axs[1].set_ylim(axs[1].get_ylim()[::-1])
    
    plt.tight_layout()
    plt.show()
    return

In [ ]:
thumbsDir = saveDir
names = [name for name in os.listdir(thumbsDir) if '.tiff' in name]
len(names)

In [ ]:
if False:
    for ifname, fname in enumerate(names[:]):
        if fname[-len('.tiff'):]=='.tiff':
            print(ifname, '\t', fname[:23], end='\t')
            prepInfoJSON(thumbsDir, 'HP', fname, eps=40, efx=0.25, efy=0.25, coords=None, min_samples=100,
                        further_thumb_downsample_facator=5, f=0.25)

# Generate table of ROIs and inspect ROIs

In [ ]:
jnames = sorted([name for name in os.listdir(thumbsDir) if ('.json' in name) and (not 'magnification' in name)])
print(len(jnames))

In [ ]:
excluded_oid = []
excluded_oid = [v+'.json' for v in excluded_oid]

In [ ]:
mpps = []
dims = []
smpp = 2.526e-07
for jname in jnames[:]:
    if not jname in excluded_oid:
        id, oid = jname.split('.oid')
        oid = oid.split('.')[0]
        with open(thumbsDir + f'magnification_dimensions_{id}.json', 'r') as tempfile:
            info = json.loads(tempfile.read())
        mpp = smpp
            
        dims.append([id, oid, mpp])
    
df_dims = pd.DataFrame(dims, columns=['id', 'oid', 'mpp'])
se = df_dims['id'].copy()
df_dims['sample'] = df_dims['id'].str.replace(' ', '_') + '.oid' + df_dims['oid'].astype(str)
df_dims['grid'] = ''
df_dims['fastq'] = ''
df_dims['roifile'] = saveDir + df_dims['id'] + '.oid' + df_dims['oid'].astype(str) + '.json'
df_dims['image'] = dataDir + df_dims['id'] + '.tif'
df_dims['case'] = se.values
df_dims = df_dims[['sample', 'fastq', 'image', 'grid', 'roifile', 'mpp', 'case']]

for sample in df_dims.index:
    p = df_dims.loc[sample]['image']
    assert os.path.isfile(p)

a = []
for roif in df_dims['roifile']:
    with open(roif, 'r') as tempfile:
        d = json.loads(tempfile.read())
        a.append(d['0']['size'] * d['0']['size'])
df_dims['area'] = a
df_dims['rank'] = df_dims.groupby('case')['area'].rank('first', ascending=False).astype(int)

apx = []
for case in df_dims['case']:
    with open(f'{saveDir}magnification_dimensions_{case}.json', 'r') as tempfile:
        d = json.loads(tempfile.read())
        apx.append(d['x'] * d['y'])
        mpps.append(smpp)
df_dims['apx'] = apx

print(len(df_dims))

In [ ]:
def prepInfoJSON2(tdatapath, model, fname, eps=50, min_samples=500, selobj=None, f=1.5, efx=0.35, efy=0.35, manshift=[0,0,0,0],
                 thumb_factor=100, further_thumb_downsample_facator=3, pen_marked=0, coords=None, do_aoi=True, infodata=None):

    id = fname[:-len('.png')]
    
    img_RGB_high_res = plt.imread(tdatapath + fname)
    
    thumb_factor *= further_thumb_downsample_facator
    img_RGB_high_res  = img_RGB_high_res[::further_thumb_downsample_facator, ::further_thumb_downsample_facator, :]
    
    print(img_RGB_high_res.shape, end='\t')

    fx = img_RGB_high_res.shape[1]/thumb_factor
    fy = img_RGB_high_res.shape[0]/thumb_factor
    fig, axs = plt.subplots(1, 2, figsize=(fx*15, fy*4))

    axs[0].imshow(img_RGB_high_res[:, :, :3], origin='lower')
    axs[0].set_xticks([])
    axs[0].set_xticklabels([])
    axs[0].set_yticks([])
    axs[0].set_yticklabels([])
    axs[0].invert_yaxis()
    axs[0].set_aspect('equal')
    
    if do_aoi:
        v = img_RGB_high_res[:, :, :3].copy()
        v = v.mean(axis=2)
        vc = v.copy()
        v[vc<100] = 255/255
        v[vc>200] = 255/255
        v[(vc>=100/255) & (vc<=200/255)] = 0
        v = pd.DataFrame(v.T).replace({255/255: np.nan}).stack().dropna().index.to_frame().values

        np.random.seed(0)
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(v)
        labels = db.labels_
        vco = pd.Series(labels).value_counts()

        axs[1].scatter(v.T[0], v.T[1], c=labels, s=1)
        for l in vco.index:
            if l!=-1:
                axs[1].text(np.median(v.T[0][np.where(labels==l)[0]]),
                         np.median(v.T[1][np.where(labels==l)[0]]),
                         l, va='center', ha='center', fontsize=20)

        print()
        objects = sorted(list(set(vco.to_dict().keys()).difference([-1])))
        res = dict()
        for l in objects:
            res[l] = dict()
            res[l]['oid'] = l
            res[l]['id'] = fname
            res[l]['block'] = ''
            for i in range(2):
                res[l][i] = dict()
                se_temp = pd.Series(v.T[i][np.where(labels==l)[0]])
                a, b = se_temp.quantile(0.05), se_temp.quantile(0.95)
                sp = b - a

                ef = efx if i==0 else efy
                sa, sb = (manshift[0], manshift[1]) if i==0 else (manshift[2], manshift[3])
                a -= (ef+sa)*sp
                b += (ef+sb)*sp

                d = img_RGB_high_res.shape[1 if i==0 else 0]
                a, b = a/d, b/d - a/d
                a = max(a, 0)
                if a + b > 1:
                    b = 1 - a
                res[l][i]['location'] = a
                res[l][i]['size'] = b

            x1 =  res[l][0]['location']*img_RGB_high_res.shape[1]
            x2 =  x1 + res[l][0]['size']*img_RGB_high_res.shape[1]
            y1 =  res[l][1]['location']*img_RGB_high_res.shape[0]
            y2 =  y1 + res[l][1]['size']*img_RGB_high_res.shape[0]
            axs[1].plot([x1, x1, x2, x2, x1], [y1, y2, y2, y1, y1], linewidth=0.5, c='crimson')

            with open(tdatapath + id + f'.oid{l}.json', 'w') as outfile:
                outfile.write(json.dumps(res[l]))
            if res[l][0]['size']*res[l][1]['size']>0.1:
                print(res[l])
    
        axs[1].set_xticks([])
        axs[1].set_xticklabels([])
        axs[1].set_yticks([])
        axs[1].set_yticklabels([])
        axs[1].invert_yaxis()
        axs[1].set_aspect('equal')
    elif infodata is not None:
        x1 =  infodata[0]['location']*img_RGB_high_res.shape[1]
        x2 =  x1 + infodata[0]['size']*img_RGB_high_res.shape[1]
        y1 =  infodata[1]['location']*img_RGB_high_res.shape[0]
        y2 =  y1 + infodata[1]['size']*img_RGB_high_res.shape[0]
        axs[0].plot([x1, x1, x2, x2, x1], [y1, y2, y2, y1, y1], linewidth=1.0, c='crimson')
        axs[1].axis('off')
        print(infodata['block'])
    else:
        axs[1].axis('off')
    
    
    plt.tight_layout()
    plt.show()
    
    return

In [ ]:
if False:
    selected = []
    for ijname, jname in enumerate(sorted(jnames)):
        if not jname in excluded_oid:
            print('%s of %s' % (ijname, len(jnames)), '\t', jname, end='\t')
            with open(thumbsDir + jname, 'r') as tempfile:
                info = json.loads(tempfile.read())
            info[0] = info['0']
            info[1] = info['1']
            info['block'] = 'None'
            se = df_dims.set_index('sample').loc[jname.split('.json')[0]]
            print(np.round((se['apx'] * se['area']) / 10**9, 2), end='\t')
            prepInfoJSON2(thumbsDir, 'HP', info['id'] + '.tiff', 
                          further_thumb_downsample_facator=5, do_aoi=False, infodata=info)
            selected.append(jname.split('.json')[0])
            print()
    print(len(selected))

In [ ]:
dfsamples = df_dims.drop(['case', 'area', 'rank', 'apx'], axis=1)
dfsamples.to_csv('samplesheet.csv', index=False)